In [1]:
import librosa
import numpy as np
import os
import math
import operator
from sklearn.cluster import KMeans
import hmmlearn.hmm
import functionMFCC as func

C:\Users\PhuongDong\miniconda3\envs\voice\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\PhuongDong\miniconda3\envs\voice\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [3]:
class_names = ["nguoi", "viet_nam", "nha", "cua", "khong", "test_nguoi", "test_viet_nam", "test_nha", "test_cua", "test_khong"]
dataset = {}
for cname in class_names:
    print(f"Load {cname} dataset")
    dataset[cname] = func.get_class_data(os.path.join("data_hmm", cname))
# Get all vectors in the datasets
all_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in dataset.items()], axis=0)
# print("vectors", all_vectors.shape)
# # Run K-Means algorithm to get clusters
# kmeans = func.clustering(all_vectors)
# print("centers", kmeans.cluster_centers_.shape)
# for cname in class_names:
#     dataset[cname] = list([kmeans.predict(v).reshape(-1,1) for v in dataset[cname]])

Load nguoi dataset
Load viet_nam dataset
Load nha dataset
Load cua dataset
Load khong dataset
Load test_nguoi dataset
Load test_viet_nam dataset
Load test_nha dataset
Load test_cua dataset
Load test_khong dataset


In [4]:
def train_model(train_set, n_component, startprob):
    model = hmmlearn.hmm.GMMHMM(n_components=n_component, n_mix=2, n_iter=1000, params='mtcw', init_params='mt', verbose=True)
    model.startprob_ = startprob
    X = np.concatenate(train_set)
    lengths = list([len(x) for x in train_set])
    # fit dat 
    print(X.shape, lengths, len(lengths))
    model.fit(X)
    return model

In [5]:
startprob6 = np.array([0.6, 0.3, 0.1, 0.0, 0.0, 0.0])
startprob9 = np.array([0.6, 0.3, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
startprob18 = np.array([0.6, 0.3, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
# transmat6 = np.array([[0.05 for i in range(6)] for j in range(6)])
# transmat12 = np.array([[0.05 for i in range(12)] for j in range(12)])
# for x in range(5):
#     transmat6[x][x] = 0.4
#     transmat6[x][x+1] = 0.4
# transmat6[5][5] = 0.75
# for x in range(11):
#     transmat12[x][x] = 0.25
#     transmat12[x][x+1] = 0.25
# transmat12[11][11] = 0.45
# print(transmat6)
models = {}
models["khong"] = train_model(dataset["khong"], 9, startprob9)
models["nha"] = train_model(dataset["nha"], 6, startprob6)
models["cua"] = train_model(dataset["cua"], 6, startprob6)
models["nguoi"] = train_model(dataset["nguoi"], 9, startprob9)
models["viet_nam"] = train_model(dataset["viet_nam"], 18, startprob18)

(1887, 36) [25, 21, 24, 27, 25, 21, 18, 24, 24, 21, 24, 29, 15, 31, 19, 21, 20, 25, 29, 32, 31, 23, 18, 35, 23, 21, 24, 28, 33, 28, 34, 23, 17, 17, 17, 23, 25, 25, 22, 25, 28, 22, 24, 19, 17, 21, 19, 23, 19, 35, 26, 24, 27, 21, 20, 23, 20, 21, 24, 16, 25, 19, 26, 21, 22, 26, 27, 19, 17, 35, 28, 22, 25, 29, 31, 21, 19, 21, 13, 25] 80


         1     -193724.3872             +nan
         2     -179682.7615      +14041.6257
         3     -177276.5145       +2406.2470
         4     -176479.2692        +797.2453
         5     -176226.9208        +252.3484
         6     -176088.0190        +138.9018
         7     -176008.1474         +79.8715
         8     -175918.9659         +89.1816
         9     -175833.2417         +85.7242
        10     -175782.9108         +50.3309
        11     -175756.8714         +26.0393
        12     -175740.2214         +16.6500
        13     -175729.4688         +10.7526
        14     -175717.7138         +11.7550
        15     -175705.9437         +11.7701
        16     -175692.8859         +13.0578
        17     -175657.1165         +35.7693
        18     -175642.4542         +14.6623
        19     -175634.0906          +8.3636
        20     -175624.3100          +9.7806
        21     -175614.4751          +9.8348
        22     -175605.6473          +8.8278
        23

(2541, 36) [21, 22, 30, 23, 17, 23, 28, 47, 32, 24, 42, 32, 33, 36, 25, 20, 25, 45, 22, 31, 28, 17, 42, 27, 19, 20, 33, 16, 36, 31, 39, 17, 33, 31, 28, 35, 32, 38, 18, 43, 29, 25, 22, 25, 21, 24, 32, 23, 62, 33, 22, 18, 33, 22, 16, 18, 54, 46, 52, 45, 22, 40, 32, 25, 26, 41, 32, 43, 43, 43, 26, 59, 40, 45, 32, 50, 47, 59, 32, 21] 80


         1     -246603.4359             +nan
         2     -231436.6428      +15166.7931
         3     -228079.7232       +3356.9196
         4     -185750.1983      +42329.5248
         5     -185349.3658        +400.8325
         6     -185175.7345        +173.6313
         7     -185050.0473        +125.6872
         8     -184939.6876        +110.3598
         9     -184880.1777         +59.5099
        10     -184824.7864         +55.3913
        11     -184778.8355         +45.9509
        12     -184744.8657         +33.9697
        13     -184704.4779         +40.3879
        14     -184635.1627         +69.3152
        15     -184538.5765         +96.5862
        16     -184452.3995         +86.1770
        17     -184411.7827         +40.6168
        18     -184384.6280         +27.1547
        19     -184367.4647         +17.1633
        20     -184348.7353         +18.7294
        21     -184309.7571         +38.9782
        22     -184205.6089        +104.1483
        23

(1872, 36) [55, 18, 24, 23, 17, 20, 22, 32, 25, 21, 20, 62, 33, 15, 18, 15, 17, 16, 18, 16, 18, 17, 62, 20, 23, 23, 48, 19, 22, 20, 29, 18, 55, 90, 62, 22, 17, 31, 18, 16, 39, 29, 26, 18, 76, 17, 12, 12, 26, 23, 23, 23, 18, 14, 16, 19, 20, 24, 16, 17, 12, 12, 26, 23, 23, 14, 24, 16, 20, 24, 16, 23, 18, 18, 18] 75


         1     -202339.0301             +nan
         2     -191456.9985      +10882.0315
         3     -189909.7071       +1547.2914
         4     -189194.6413        +715.0658
         5     -188700.2421        +494.3992
         6     -188481.2596        +218.9825
         7     -188400.5418         +80.7178
         8     -188304.8161         +95.7256
         9     -188184.2242        +120.5919
        10     -188127.2477         +56.9766
        11     -188071.1838         +56.0639
        12     -188015.1864         +55.9974
        13     -187974.3335         +40.8529
        14     -187894.8655         +79.4680
        15     -187848.6095         +46.2560
        16     -187837.4551         +11.1544
        17     -187823.2680         +14.1871
        18     -187807.7984         +15.4696
        19     -187797.8316          +9.9668
        20     -187789.6968          +8.1348
        21     -187785.5199          +4.1769
        22     -187781.7112          +3.8087
        23

(2020, 36) [29, 30, 15, 43, 30, 30, 14, 25, 24, 30, 35, 23, 17, 24, 21, 31, 22, 28, 29, 28, 19, 16, 16, 31, 24, 22, 19, 13, 21, 23, 30, 19, 32, 47, 18, 34, 26, 21, 10, 23, 29, 25, 21, 25, 25, 12, 27, 24, 37, 22, 26, 29, 17, 26, 24, 33, 36, 22, 21, 30, 31, 31, 22, 22, 26, 23, 36, 32, 23, 17, 32, 28, 21, 32, 21, 16, 25, 26, 35, 18] 80


         1     -211817.3497             +nan
         2     -198647.9097      +13169.4400
         3     -196514.2111       +2133.6986
         4     -195946.9642        +567.2468
         5     -195639.8691        +307.0951
         6     -195518.1550        +121.7141
         7     -195432.0230         +86.1320
         8     -195338.5885         +93.4345
         9     -195287.9327         +50.6558
        10     -195241.1141         +46.8186
        11     -195206.1973         +34.9168
        12     -195184.7416         +21.4557
        13     -195173.7867         +10.9549
        14     -195168.1741          +5.6126
        15     -195159.7488          +8.4253
        16     -195146.7889         +12.9599
        17     -195133.2279         +13.5609
        18     -195123.9161          +9.3119
        19     -195114.6427          +9.2734
        20     -195102.5362         +12.1065
        21     -195093.4738          +9.0623
        22     -195082.2901         +11.1837
        23

(3536, 36) [52, 43, 35, 55, 34, 55, 47, 51, 41, 49, 34, 58, 41, 41, 75, 30, 50, 53, 39, 42, 40, 42, 49, 40, 39, 36, 40, 41, 54, 47, 46, 55, 38, 49, 37, 35, 45, 39, 58, 42, 40, 43, 34, 31, 47, 48, 37, 58, 38, 37, 50, 41, 37, 43, 50, 49, 48, 44, 51, 32, 42, 51, 42, 37, 30, 50, 45, 35, 39, 55, 47, 45, 39, 40, 61, 56, 42, 40, 46, 39] 80


         1     -365699.2819             +nan
         2     -332911.5292      +32787.7527
         3     -328071.6563       +4839.8729
         4     -326878.2420       +1193.4143
         5     -326426.0044        +452.2376
         6     -326107.9336        +318.0708
         7     -325829.7720        +278.1617
         8     -325554.4277        +275.3442
         9     -325348.8471        +205.5806
        10     -325145.1727        +203.6744
        11     -324999.6194        +145.5533
        12     -324860.8607        +138.7587
        13     -324700.9345        +159.9261
        14     -324585.1858        +115.7487
        15     -324498.1624         +87.0234
        16     -324440.4260         +57.7364
        17     -324384.5499         +55.8761
        18     -324298.7162         +85.8337
        19     -324240.2636         +58.4526
        20     -324217.9074         +22.3562
        21     -324208.2385          +9.6689
        22     -324199.5437          +8.6948
        23

In [6]:
print("Testing")
dataset["test_nha"] = func.get_class_data(os.path.join("data_hmm", "test_nha"))
for true_cname in class_names:
    if true_cname[:4] == "test":
        result = 0
        length = 0
        for O in dataset[true_cname]:
            length+=1
            score = {cname: model.score(O, [len(O)]) for cname, model in models.items()}
            if true_cname[5:] == max(score.items(), key=operator.itemgetter(1))[0]:
                result = result + 1
            else: 
                print(true_cname, score, length, max(score.items(), key=operator.itemgetter(1))[0])
        print('Acc ' + true_cname[5:] , result/length, result, length)

Testing


Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance


Acc nguoi 1.0 20 20


Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance


Acc viet_nam 1.0 20 20


...

Acc nha 1.0 20 20


Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance


Acc cua 1.0 20 20


Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance


Acc khong 1.0 20 20


In [7]:
import pickle

In [8]:
with open("modelHMM.pkl", "wb") as file:
    pickle.dump(models, file)